# **데이터 API(기초, CSV 파일 기준)**


In [1]:
import tensorflow as tf

X = tf.range(10)  # 데이터 텐서(그냥 예시)
dataset = tf.data.Dataset.from_tensor_slices(X) #데이터셋 만들어짐
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [2]:
#순회하여 확인 가능
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [5]:
# 이렇게 만들 수도 있다.
X_nested = {"a": ([1, 2, 3], [4, 5, 6]), "b": [7, 8, 9]}
dataset = tf.data.Dataset.from_tensor_slices(X_nested)
for item in dataset:
    print(item)

{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=4>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=7>}
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=5>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=8>}
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=6>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=9>}


In [12]:
#연쇄 변환
dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))
#0~9까지 3번 순회하고 길이는 7로 자른다.
dataset = dataset.repeat(3).batch(7) #batch(7, drop_remainder = True)로 호출하면 밑에 2개는 자름
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [24]:
dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))
dataset = dataset.map(lambda x: x ** 2)  # x는 하나의 배치이다.
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(36, shape=(), dtype=int32)
tf.Tensor(49, shape=(), dtype=int32)
tf.Tensor(64, shape=(), dtype=int32)
tf.Tensor(81, shape=(), dtype=int32)


In [27]:
dataset = dataset.filter(lambda x: tf.reduce_sum(x) > 20) #이렇게 필터링도 가능
for item in dataset:
    print(item)

tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(36, shape=(), dtype=int32)
tf.Tensor(49, shape=(), dtype=int32)
tf.Tensor(64, shape=(), dtype=int32)
tf.Tensor(81, shape=(), dtype=int32)


In [31]:
for item in dataset.take(3): # 위에서 몇개만 짜를수도 있다.
    print(item)

tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(36, shape=(), dtype=int32)
tf.Tensor(49, shape=(), dtype=int32)


In [ ]:
"""
데이터 Shuffle
예를 들어 왼쪽에 카드 Deck 뭉치가 있을때 카드 뭉치에서 buffer_size개만큼 카드를 뽑고
오른쪽에 하나 내려놓고, 다시 왼쪽에서 하나 뽑고..를 반복해서 오른쪽에 카드 Deck을 만드는 것이다.
이것은 buffer_size가 작을수록 완전한 Shuffle이 이루어지지 않는다는 것을 시사한다.

이를 해결하는 방법
1. 버퍼를 늘린다(데이터셋이 너무 크면 불가능)
2. 원본 샘플을 섞는다(데이터셋이 커도 가능.)
"""
dataset = tf.data.Dataset.range(10).repeat(2) #[0,1,2,3,4...,0,1,2,3,4...]

dataset = dataset.shuffle(buffer_size=4, seed=42).batch(7)
for item in dataset:
    print(item)

"""
dataset = tf.data.Dataset.range(10).repeat(2) #[0,1,2,3,4...,0,1,2,3,4...]
dataset = dataset.shuffle(buffer_size=4, reshuffle_each_iteration = False,
                          seed=42).repeat(3).batch(7)
for item in dataset:
    print(item)
과 같이 쓰면 shuffle을 x3번 한 배열을 생성하는데, 그 배열들 순서가 다 똑같이 나옴
"""


**그렇다면 원본 샘플은 어떻게 섞을까?**

1. 먼저 데이터 셋을 불러오고

```
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import numpy as np
from pathlib import Path

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)
```
2. 데이터셋을 분할한다(아래는 20개로)

```
def save_to_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = Path() / "datasets" / "housing"
    housing_dir.mkdir(parents=True, exist_ok=True)
    filename_format = "my_{}_{:02d}.csv"

    filepaths = []
    m = len(data)
    chunks = np.array_split(np.arange(m), n_parts)
    for file_idx, row_indices in enumerate(chunks):
        part_csv = housing_dir / filename_format.format(name_prefix, file_idx)
        filepaths.append(str(part_csv))
        with open(part_csv, "w") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = save_to_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_csv_files(test_data, "test", header, n_parts=10)
```

3. 그러면 파일이 20개로 분할된 것을 볼 수 있고

```
train_filepaths
->['datasets/housing/my_train_00.csv',
'datasets/housing/my_train_01.csv',
'datasets/housing/my_train_02.csv',
'datasets/housing/my_train_03.csv',
'datasets/housing/my_train_04.csv',
'datasets/housing/my_train_05.csv',
'datasets/housing/my_train_06.csv',
                [...]
'datasets/housing/my_train_15.csv',
'datasets/housing/my_train_16.csv',
'datasets/housing/my_train_17.csv',
'datasets/housing/my_train_18.csv',
'datasets/housing/my_train_19.csv']
```

4. 파일 경로를 섞은 다음...

```
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)
for filepath in filepath_dataset:
    print(filepath)
 ->
tf.Tensor(b'datasets/housing/my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_17.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_00.csv', shape=(), dtype=string)
                                 [...]
tf.Tensor(b'datasets/housing/my_train_18.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_04.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_06.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_03.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_08.csv', shape=(), dtype=string)
```

5.마지막으로 순서대로 5개를 불러올 수 있다.   
파일 5개를 불러오고, 순서대로 한 줄씩 읽고, 다 읽으면 5개 불러오는 식이다.   
* interleave는 (num_parallel_calls = n) 매개변수로 병렬화를 지원한다.   
* TextLineDataset에 파일 경로 리스트를 전달할수도 있지만 파일을 섞거나   
헤더를 (skip(1))을 건너뛰지 못한다.

```
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1), #첫번째 줄은 열 이름
    cycle_length=n_readers)

 for line in dataset.take(5):
    print(line)
 ->tf.Tensor(b'4.5909,16.0,5.475877192982456,1.0964912280701755,1357.0,2.9758771929824563,33.63,-117.71,2.418', shape=(), dtype=string)
tf.Tensor(b'2.4792,24.0,3.4547038327526134,1.1341463414634145,2251.0,3.921602787456446,34.18,-118.38,2.0', shape=(), dtype=string)
tf.Tensor(b'4.2708,45.0,5.121387283236994,0.953757225433526,492.0,2.8439306358381504,37.48,-122.19,2.67', shape=(), dtype=string)
tf.Tensor(b'2.1856,41.0,3.7189873417721517,1.0658227848101265,803.0,2.0329113924050635,32.76,-117.12,1.205', shape=(), dtype=string)
tf.Tensor(b'4.1812,52.0,5.701388888888889,0.9965277777777778,692.0,2.4027777777777777,33.73,-118.31,3.215', shape=(), dtype=string)
```
\
\
**데이터 전처리와 적재 합치기**
- list_file -> interleave -> map(preprocess) -> shuffle -> batch -> prepatch

```
앞에서 만든 데이터(적재한 데이터)들을 표준화 전처리와 함께 묶은 예시
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_mean, X_std = scaler.mean_, scaler.scale_  # 추가 코드
n_inputs = 8

def parse_csv_line(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    return tf.stack(fields[:-1]), tf.stack(fields[-1:])

def preprocess(line):
    x, y = parse_csv_line(line)
    return (x - X_mean) / X_std, y


def csv_reader_dataset(filepaths, n_readers=5, n_read_threads=None,
                       n_parse_threads=5, shuffle_buffer_size=10_000, seed=42,
                       batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths, seed=seed)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size, seed=seed)
    return dataset.batch(batch_size).prefetch(1) #cpu가 다음 배치 준비
```
```
그럼 이제
train_set = csv_reader_dataset(train_filepaths)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

X_train,y_train 대신 train_set를
X_valid, y_valid 대신 valid_set를
X_test, y_test 대신 test_set를 사용할 수 있다.

-> 이런 방식을 tf.data pipeline에서 전처리하는 방식이라고 한다.
```

In [ ]:
"""
tf.data.Dataset 클래스 메소드 정리
* : 저자 추천
* apply(): 이 데이터셋에 변환 함수를 적용합니다.
as_numpy_iterator(): 데이터셋의 모든 요소를 넘파이(numpy)로 변환하는 반복자를 반환합니다.
batch(): 이 데이터셋의 연속적인 요소들을 배치로 결합합니다.
bucket_by_sequence_length(): 데이터셋의 요소를 길이에 따라 분류하는 변환을 수행합니다.
cache(): 이 데이터셋의 요소들을 캐시합니다.
cardinality(): 데이터셋의 카디널리티(요소의 수)를 반환합니다(알려진 경우).
choose_from_datasets(): datasets에서 요소를 결정적으로 선택하는 데이터셋을 생성합니다.
* concatenate(): 주어진 데이터셋을 이 데이터셋과 연결하여 새로운 Dataset을 생성합니다.
counter(): start부터 step 크기의 단계로 셈하는 Dataset을 생성합니다.
element_spec(): 이 데이터셋의 요소의 타입 명세를 반환합니다.
enumerate(): 이 데이터셋의 요소들을 열거합니다.
filter(): predicate에 따라 이 데이터셋을 필터링합니다.
* flat_map(): 이 데이터셋에 map_func을 매핑하고 결과를 평탄화합니다.
* from_generator(): generator에 의해 생성된 요소들로 Dataset을 생성합니다. (deprecated arguments)
from_tensor_slices(): 주어진 텐서의 슬라이스로 Dataset을 생성합니다.
* from_tensors(): 주어진 텐서들로 구성된 단일 요소를 가진 Dataset을 생성합니다.
get_single_element(): dataset의 단일 요소를 반환합니다.
group_by_window(): 키에 따라 요소의 윈도우를 그룹화하고 그것들을 축소합니다.
ignore_errors(): 오류를 유발하는 요소들을 무시합니다.
interleave(): 이 데이터셋에 map_func을 매핑하고 결과를 교차 배열합니다.
list_files(): 하나 이상의 글로브 패턴에 일치하는 모든 파일의 데이터셋입니다.
load(): 이전에 저장된 데이터셋을 불러옵니다.
map(): 이 데이터셋의 요소들에 map_func을 매핑합니다.
options(): 이 데이터셋과 그 입력의 옵션을 반환합니다.
* padded_batch(): 이 데이터셋의 연속적인 요소들을 패딩된 배치로 결합합니다.
* prefetch(): 이 데이터셋에서 요소들을 미리 가져오는 Dataset을 생성합니다.
ragged_batch(): 이 데이터셋의 연속적인 요소들을 tf.RaggedTensor들로 결합합니다.
random(): 의사난수 값을 가진 Dataset을 생성합니다.
range(): 값의 범위에 대한 Dataset을 생성합니다.
rebatch(): 이 데이터셋의 요소들을 재배치하는 Dataset을 생성합니다.
* reduce(): 입력 데이터셋을 단일 요소로 축소합니다.
rejection_resample(): 목표 분포에 도달하기 위해 요소들을 재샘플링합니다.
repeat(): 이 데이터셋을 반복하여 각 원본 값을 count 번씩 보여줍니다.
sample_from_datasets(): datasets의 데이터셋에서 무작위로 요소를 샘플링합니다.
save(): 주어진 데이터셋의 내용을 저장합니다.
scan(): 입력 데이터셋에 대해 함수를 스캔하는 변환입니다.
* shard(): 이 데이터셋의 1/num_shards만 포함하는 Dataset을 생성합니다.
shuffle(): 이 데이터셋의 요소들을 무작위로 섞습니다.
skip(): 이 데이터셋에서 count 요소를 건너뛰는 Dataset을 생성합니다.
snapshot(): 입력 데이터셋의 출력을 유지하기 위한 API입니다.
sparse_batch(): 연속적인 요소들을 tf.sparse.SparseTensor들로 결합합니다.
take(): 이 데이터셋에서 최대 count 요소를 가진 Dataset을 생성합니다.
take_while(): predicate에 따라 데이터셋 반복을 중단하는 변환입니다.
unbatch(): 데이터셋의 요소들을 여러 요소로 분할합니다.
unique(): Dataset의 중복 요소를 제거하는 변환입니다.
* window(): "윈도우"의 데이터셋을 반환합니다.
with_options(): 주어진 옵션을 설정한 새로운 tf.data.Dataset을 반환합니다.
* zip(): 주어진 데이터셋들을 함께 묶어 Dataset을 생성합니다.
"""

In [ ]:
"""
중요 함수 요약
apply()
기능: 사용자 정의 또는 미리 정의된 변환 함수를 데이터셋에 적용합니다.
사용 예: 데이터셋의 구조를 변경하거나, 복잡한 변환을 데이터셋에 적용할 때 유용합니다.

concatenate()
기능: 두 데이터셋을 순차적으로 연결합니다. 첫 번째 데이터셋 뒤에 두 번째 데이터셋이 이어집니다.
사용 예: 서로 다른 두 데이터셋을 하나의 연속된 데이터셋으로 병합할 때 사용됩니다.

flat_map()
기능: 각 요소에 함수를 적용하고, 그 결과를 평탄화하여 하나의 데이터셋으로 반환합니다.
사용 예: 각 요소가 여러 개의 하위 요소를 포함하는 경우, 이를 단일 데이터셋으로 평탄화하는 데 사용됩니다.

from_generator()
기능: 파이썬 제너레이터(generator) 함수로부터 데이터셋을 생성합니다.
사용 예: 파이썬의 반복 가능한 객체나 함수를 통해 데이터를 생성할 때 사용됩니다.

from_tensors()
기능: 텐서들로부터 단일 요소를 가진 데이터셋을 생성합니다.
사용 예: 고정된 텐서 데이터를 데이터셋으로 변환할 때 사용됩니다.

padded_batch()
기능: 배치로 결합된 요소들을 패딩하여 모든 요소가 같은 크기를 갖도록 합니다.
사용 예: 길이가 다른 시퀀스 데이터를 동일한 길이로 패딩하여 배치 처리할 때 사용됩니다.

prefetch()
기능: 데이터 처리 및 모델 학습 파이프라인의 효율성을 높이기 위해 데이터셋의 요소들을 미리 가져옵니다.
사용 예: 학습 중에 데이터 로딩 시간을 줄이고 GPU와 같은 계산 자원의 활용도를 높일 때 사용됩니다.

reduce()
기능: 데이터셋을 단일 요소로 축소합니다. 주어진 리듀스 함수에 따라 모든 요소를 결합합니다.
사용 예: 데이터셋의 모든 요소를 합치거나 평균을 내는 등의 집계 연산에 사용됩니다.

shard()
기능: 전체 데이터셋을 여러 개의 샤드로 분할하고, 지정된 샤드만 포함하는 데이터셋을 생성합니다.
사용 예: 분산 학습 환경에서 데이터셋을 여러 작업자 간에 나눌 때 사용됩니다.

window()
기능: 데이터셋을 여러 개의 작은 '윈도우' 데이터셋으로 분할합니다.
사용 예: 시계열 데이터나 연속적인 데이터 시퀀스를 처리할 때 각 시퀀스를 작은 부분으로 나누는 데 사용됩니다.

zip()
기능: 둘 이상의 데이터셋을 병렬적으로 결합하여, 요소들이 튜플 형태로 묶인 새로운 데이터셋을 생성합니다.
사용 예: 서로 다른 특성을 가진 여러 데이터셋을 동시에 사용할 때 사용됩니다. 예를 들어, 이미지 데이터셋과 해당 이미지의 레이블 데이터셋을 결합할 때 유용합니다.
"""

# **TFTRecord**

In [ ]:
"""
TFTRecord는 이진 레코드 목록으로, 대용량 데이터를 저장하고 효율적으로 읽기 위해
텐서플로에서 선호하는 포맷이다. CSV나 다른 포맷에서 병목이 생기거나 문제가 없다면
TFTRecord를 사용하지 않아도 된다.
"""
import tensorflow as tf

#아래와 같이 파일을 만들 수 있다.
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"This is the first record")
    f.write(b"And this is the second record")

#파일은 이렇게 불러온다.
filepaths = ["my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

In [ ]:
#파일을 압축하거나, 압축한 파일을 불러오려면 아래와 같이 해야한다.

#압축 형식을 지정하고, 파일을 만든다.
options = tf.io.TFRecordOptions(compression_type="GZIP")
with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f:
    f.write(b"Compress, compress, compress!")

#불러올때는 압축 형식을 지정하고 불러온다.
dataset = tf.data.TFRecordDataset(["my_compressed.tfrecord"],
                                  compression_type="GZIP")
for item in dataset:
    print(item)

# **전처리** (number, string, image)


**1. Normlization (수치 데이터)**
```
#훈련 전에 데이터에 전처리 처리
norm_layer = tf.keras.layers.Normalization()
nrom_layer.adapt(X_train)
X_train_scaled = norm_layer(X_train)
X_valid_scaled = norm_layer(X_valid)

# 모델 훈련
model = keras.model.Sequential([layers.Dense(1)])
model.complie = [...]
model.fit(X_train_scaled, y_train, epoch = 5,
          validation_data = (X_valid_scaled, y_valid))

# 최종 모델 테스트
final_model = keras.Sequential([norm_layer, model])
X_new = X_test[:3]
y_pred = final_model(X_new)
```

**2. Discretization (수치 데이터)**   
값을 구간 별로 나누는것 (20대, 30대, 40대 ...)
```
age = tf.constant([[10.], [93.], [57.], [18.], [37.], [5.]]).

# 18보다 작은 값, 18~50 값, 50~ 값이 0,1,2로 인코딩 된다.
discretize_layer = tf.keras.layers.Discretization(bin_boundaries=[18., 50.])
age_categories = discretize_layer(age)
-> array([[0],
       [2],
       [2],
       [1],
       [1],
       [0]])>

#아니면 이렇게 원하는 구간 개수를 전달할 수도 있다.
discretize_layer = tf.keras.layers.Discretization(num_bins=3)
discretize_layer.adapt(age)
age_categories = discretize_layer(age)
->array([[1],
       [2],
       [2],
       [1],
       [2],
       [0]])>

#그러나 이렇게 구간별로 나눈다음 써먹으려면 원핫인코딩으로 변환해야 한다.
onehot_layer = tf.keras.layers.CategoryEncoding(num_tokens=3)
onehot_layer(age_categories)
->array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)>

#이렇게 쓰면 멀티-핫 인코딩을 수행할 수 있으며, 원-핫 인코딩과
멀티-핫 인코딩중 뭐가 더 나을지는 해봐야 안다.
two_age_categories = np.array([[1, 0], [2, 2], [2, 0]])
onehot_layer(two_age_categories)
```

**3.StringLookup (문자 데이터)**
```
#output_mode = "one_hot"으로 지정 안하면 정수 인덱스를 반환함.
str_lookup_layer = tf.keras.layers.StringLookup(output_mode="one_hot")
str_lookup_layer.adapt(cities)
str_lookup_layer([["Paris"], ["Auckland"], ["Auckland"], ["Montreal"]])
->array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]], dtype=float32)>
```

**4.Embedding (문자 데이터)**
```
만약 특성이 수 만개가 넘어가면 이걸 모두 원-핫 인코딩 하는것은 불가능하다.
50000개의 특성(차원)이 있을때 이를 원-핫 인코딩으로 표현하려면   
50000차원의 희소 벡터가 필요할 것이다. 그러나 임베딩을 활용하면
100차원의 밀집 벡터로 표현할 수 있다.
import tensorflow as tf

#문자열을 정수로 변환한 뒤, 임베딩하기.
ocean_prox = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]
str_lookup_layer = tf.keras.layers.StringLookup()
str_lookup_layer.adapt(ocean_prox)
lookup_and_embed = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=[], dtype=tf.string),  # WORKAROUND
    str_lookup_layer,
    tf.keras.layers.Embedding(input_dim=str_lookup_layer.vocabulary_size(),
                             output_dim=2)
])

lookup_and_embed(np.array(["<1H OCEAN", "ISLAND", "<1H OCEAN"]))
-> array([[-0.03449249,  0.01491299],
       [ 0.02290317, -0.03581462],
       [-0.03449249,  0.01491299]], dtype=float32)>
```
\

```
수치 특성과 문자 특성을 모두 전처리하는 예시
tf.random.set_seed(42)
np.random.seed(42)
X_train_num = np.random.rand(10_000, 8)
X_train_cat = np.random.choice(ocean_prox, size=10_000)
y_train = np.random.rand(10_000, 1)
X_valid_num = np.random.rand(2_000, 8)
X_valid_cat = np.random.choice(ocean_prox, size=2_000)
y_valid = np.random.rand(2_000, 1)

num_input = tf.keras.layers.Input(shape=[8], name="num")
cat_input = tf.keras.layers.Input(shape=[], dtype=tf.string, name="cat")
cat_embeddings = lookup_and_embed(cat_input)
encoded_inputs = tf.keras.layers.concatenate([num_input, cat_embeddings])
outputs = tf.keras.layers.Dense(1)(encoded_inputs)
model = tf.keras.models.Model(inputs=[num_input, cat_input], outputs=[outputs])
model.compile(loss="mse", optimizer="sgd")
history = model.fit((X_train_num, X_train_cat), y_train, epochs=5,
                    validation_data=((X_valid_num, X_valid_cat), y_valid))
```

**5. TextVectorization (텍스트 전처리)**
```
standardize = None (대소문자, 구두점 유지)
output_sequence_length = N (원하는 길이로 문자 자르거나 패딩)


#사용법은 StringLookup과 유사하다.
train_data = ["To be", "!(to be)", "That's the question", "Be, be, be."]
text_vec_layer = tf.keras.layers.TextVectorization()
text_vec_layer.adapt(train_data)
text_vec_layer(["Be good!", "Question: be or be?"])
-> array([[2, 1, 0, 0],
       [6, 2, 1, 2]])>

# TextVectorization에는 다양한 옵션이 있으며 아래와 같은
tf_idf는 많이 등장하는 단어(to, be, is)등에는 낮은 가중치를 부여하고
드물게 등장하는 단어에는 높은 가중치를 부여한다.

text_vec_layer = tf.keras.layers.TextVectorization(output_mode="tf_idf")
text_vec_layer.adapt(train_data)
text_vec_layer(["Be good!", "Question: be or be?"])

#마지막으로 tensorflow hub나 허깅페이스 허브 등에서 사전 훈련된 모델을
다운로드 받아서 사용할 수 있다.
import tensorflow_hub as hub

hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2")
sentence_embeddings = hub_layer(tf.constant(["To be", "Not to be"]))
sentence_embeddings.numpy().round(2)
```

**6. 이미지 전처리**
1. Resizing (이미지 크기 변경, 비율 고정 지원)
2. Rescaling (픽셀 스케일 조정)
3. CenterCrop (주위 이미지 Crop)
```
image = [이미지를 불러온다...]
crop_image_layer = tf.keras.layers.CaenterCrop(height = 100, width = 100)
cropped_images = crop_image_layer(images)
```